Run selection
===========

Anastasia has summarized the standalone runs of the TRD in 2021 in the Jira ticket [\[TRD-112\]](https://alice.its.cern.ch/jira/browse/TRD-112).

From this ticket, I picked run 504428 with the following comment:
```
504419, 100 hz, 42212, calrate 1; 2.3GB/s, 100000 events
504425, 1 hz, 42212, calrate 1; 151MB/s, 1000 events
504428, 10 khz, 42212, calrate 1; 151MB/s, 49944 events ; accepted rate 5 k
```

In [1]:
run = 504419
filepattern = f"/alice/data/2021/OCT/{run}/raw"

outdir = f"/eos/user/t/tdietel/data/noise/{run}"

# Path(outdir).mkdir(parents=True, exist_ok=True)
%cd $outdir

/eos/home-t/tdietel/data/noise/504419


In [12]:
from IPython.core.magic import register_cell_magic
import string
import re
import argparse
import pathlib

class NamespaceFormatter(string.Formatter):
   def __init__(self, namespace={}):
       string.Formatter.__init__(self)
       self.namespace = namespace

   def get_value(self, key, args, kwds):
       if isinstance(key, str):
           try:
               # Check explicitly passed arguments first
               return kwds[key]
           except KeyError:
               return self.namespace[key]
       else:
           string.Formatter.get_value(key, args, kwds)

            
@register_cell_magic
def genfile(line, cell):
    """ Generate file 
    
    This magic is an extension of the %%writefile magic. The additional functionality 
    includes:
      - variable expansion (similar to string.format and f-strings)
      - set file permisssions with -m | --mode flag"""

    # parse the command line arguments
    parser = argparse.ArgumentParser(description='Generate file magic')
    parser.add_argument('outfile', type=str, help='output file')
    parser.add_argument('-m', '--mode', help='file permissions')
    args = parser.parse_args(line.split())


    # format the content
    fmt = NamespaceFormatter(globals())
    content = fmt.format(cell)
    content = content.replace("\n    ", "\n\t")

    # TODO: this should check if the file content changed to
    # prevent updating the timestamp if nothing changed.
    # This will help with Makefile dependencies.
    
    # generate the file
    filename = pathlib.Path(args.outfile)
    with open(filename, "w") as out:
            out.write(content)
    if args.mode is not None:
        filename.chmod(int(args.mode,8))

    print(f"Generated file '{filename}'")

    return 

In [3]:
info = %system alien-token-info
print(info)

import alienpy.alien as alien

alien.setup_logging()
try:
    j = alien.AliEn()
#     j.ProcessMsg('whoami -v')

except SystemExit:
    print("ERROR initializing AliEn session")
    print("Make sure you have a valid token by opening a SWAN terminal and running `alien-token-init`")

['DN >>> /C=ch/O=AliEn2/CN=Users/CN=tdietel/OU=tdietel', 'ISSUER >>> /C=ch/O=AliEn2/CN=AliEn CA', 'BEGIN >>> 2022-03-30 23:39:23', 'EXPIRE >>> 2022-05-01 01:39:23']


In [4]:
import json
import collections
from pathlib import Path
from pprint import pprint

AlienFileInfo = collections.namedtuple('AlienFileInfo', ['path','size'])
results = j.run(f'ls -l {filepattern}').ansdict['results']
fileinfo = [AlienFileInfo(Path(x['path']), int(x['size'])) for x in results]
# print(fileinfo)

# OPTIONAL: restrict results
fileinfo = fileinfo[1:]

In [5]:
totalsize = sum([x.size for x in fileinfo])
print(f"Using {len(fileinfo)} files with {totalsize / float(1<<30):.3f} GiB")
print()

# for x in fileinfo: print(f"{x.size:15d}  {x.path}")
    
with open(Path(outdir)/"files.txt", "w") as f:
    for x in fileinfo:
        print(f"{x.size:15d}  {x.path}")
        f.write(str(x.path))

Using 1 files with 0.042 GiB

       45543288  /alice/data/2021/OCT/504419/raw/o2_rawtf_run00504419_tf00006252.tf


### Copying files

It's straightforward to download a file with `alien.py`

In [10]:
outdir = pathlib.Path(outdir)

for f in [ x.path for x in fileinfo]:
    if not (outdir / f.name).exists():
        j.ProcessMsg(f"cp {f} file:{outdir / f.name}")
    
%system ls -lh $outdir

['total 58M',
 'drwxr-xr-x. 2 tdietel 1395 4.0K Mar 30 17:07 bak',
 '-rwxr-xr-x. 1 tdietel 1395  242 Mar 31 12:30 convert.sh',
 '-rw-r--r--. 1 tdietel 1395  880 Mar 31 12:33 dpl-config.json',
 '-rw-r--r--. 1 tdietel 1395   66 Mar 31 13:07 files.txt',
 '-rw-r--r--. 1 tdietel 1395   90 Mar 31 12:29 Makefile',
 '-rw-r--r--. 1 tdietel 1395  44M Mar 31 13:09 o2_rawtf_run00504419_tf00006252.tf',
 '-rw-r--r--. 1 tdietel 1395  15M Mar 31 12:33 trddigits.root',
 '-rw-r--r--. 1 tdietel 1395 8.8K Mar 31 12:33 trdtracklets.root']

## Convert timeframe


In [13]:
%%genfile -m 0755 convert.sh
#!/bin/sh

eval $(alienv printenv VO_ALICE@O2::nightly-20220124-1)

o2-raw-tf-reader-workflow --input-data files.txt \
| o2-trd-datareader --fixsm1617 --tracklethcheader 2 –enable-root-output \
| o2-trd-digittracklet-writer \
--run --batch


Generated file 'convert.sh'


In [14]:
%%genfile Makefile

conv: convert.sh files.txt
    ./convert.sh

clean:
    rm -f trdtracklets.root trddigits.root

Generated file 'Makefile'


In [15]:
%system ls -l
# %system hexdump -C Makefile

['total 59358',
 'drwxr-xr-x. 2 tdietel 1395     4096 Mar 30 17:07 bak',
 '-rwxr-xr-x. 1 tdietel 1395      242 Mar 31 13:11 convert.sh',
 '-rw-r--r--. 1 tdietel 1395      880 Mar 31 12:33 dpl-config.json',
 '-rw-r--r--. 1 tdietel 1395       66 Mar 31 13:07 files.txt',
 '-rw-r--r--. 1 tdietel 1395       90 Mar 31 13:11 Makefile',
 '-rw-r--r--. 1 tdietel 1395 45543288 Mar 31 13:09 o2_rawtf_run00504419_tf00006252.tf',
 '-rw-r--r--. 1 tdietel 1395 15222345 Mar 31 12:33 trddigits.root',
 '-rw-r--r--. 1 tdietel 1395     8962 Mar 31 12:33 trdtracklets.root']